In [1]:
from qecsim.models.rotatedplanar import RotatedPlanarCode, RotatedPlanarPauli
from ModifiedRotatedPlanarRMPSDecoder import ModifiedRotatedPlanarRMPSDecoder
from deformation_transformations import noise_permutations_from_deformation
from qecsim.models.generic import BiasedDepolarizingErrorModel
import numpy as np

In [2]:

L = 21 # Surface code distance
chi = 6 # bond dimension
eta = 2 # bias
prob = 0.11 # total error rate
deformation = 3*np.ones((L, L), dtype=int)  # Clifford deformation

error_model = BiasedDepolarizingErrorModel(bias=eta, axis='Z')
print(*[f"{'IXYZ'[i]}: {p:.2e}\t" for i, p in enumerate(error_model.probability_distribution(prob))])

code = RotatedPlanarCode(L, L)
n,k,d = code.n_k_d
print(code.ascii_art())

assert code.size == deformation.shape, "Deformation shape must match code size"
permutation = noise_permutations_from_deformation(deformation)

decoder = ModifiedRotatedPlanarRMPSDecoder(chi)

I: 8.90e-01	 X: 1.83e-02	 Y: 1.83e-02	 Z: 7.33e-02	
  ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐      
  ·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·─┐
  │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░│
┌─·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·─┘
│░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│  
└─·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·─┐
  │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░│
┌─·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·─┘
│░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│  
└─·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·───·─┐
  │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░│
┌─

In [6]:
def code_pauli_error(code, pI, px, py, pz):
    """Generate a random Pauli error on the code with given probabilities."""
    n,k,d = code.n_k_d
    pauli = np.random.choice([0,1,2,3], size=n, p=[pI, px, pz, py])
    error = np.zeros((2*n), dtype=int)
    error[n:] = pauli >= 2
    error[:n] = pauli % 2 == 1
    return error

In [7]:
error = code_pauli_error(code, *error_model.probability_distribution(prob))
syndrome = code.stabilizers @ error % 2

print(code.ascii_art(pauli=RotatedPlanarPauli(code, error)))

  ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐      
  ·───·───·───Z───·───·───Z───·───·───·───·───·───·───·───Z───Z───·───·───·───·───·─┐
  │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░│
┌─·───·───·───·───·───·───·───·───·───·───·───Z───·───·───·───·───·───·───·───·───Z─┘
│░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│  
└─·───·───·───·───·───·───·───·───Z───·───·───·───·───·───·───X───Z───·───·───·───Z─┐
  │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░│
┌─·───·───·───·───Y───·───·───Z───·───·───·───·───·───·───·───Z───·───·───·───·───·─┘
│░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│  
└─·───·───·───·───Z───Z───Z───·───·───·───·───·───·───·───·───·───·───·───·───·───·─┐
  │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░│
┌─·───·───·───·───·───·───·───·───·───·───·───Z───·───

In [8]:
# recovery = decoder.decode(code, syndrome, error_model, prob, error_permutation=permutation)
recovery = decoder.decode(code, syndrome, error_model, prob)

recovery[:n], recovery[n:] = recovery[n:], recovery[:n].copy() # swap X and Z
recovery_syndromes = code.stabilizers @ recovery % 2

assert np.array_equal(syndrome, recovery_syndromes), f"{recovery_syndromes} != {syndrome}"

logicals_to_pauli = np.array([['I', 'Z'], ['X', 'Y']])
error_logical = code.logicals @ error % 2
recovery_logical = code.logicals @ recovery % 2
logical_error = (error_logical + recovery_logical) % 2
print(f"Error logical:    {logicals_to_pauli[*error_logical]:s}")
print(f"Recovery logical: {logicals_to_pauli[*recovery_logical]:s}")
print(f"Logical error:    {logicals_to_pauli[*logical_error]:s}")

print(code.ascii_art(pauli=RotatedPlanarPauli(code, recovery)))

Error logical:    X
Recovery logical: X
Logical error:    I
  ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌───┐      
  ·───·───·───Z───Z───Z───Z───·───·───·───·───·───·───·───·───·───·───·───·───·───·─┐
  │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░│
┌─·───·───·───·───Z───Z───·───·───·───·───·───·───Z───Z───·───·───Z───Z───Z───Z───·─┘
│░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│  
└─Z───Z───Z───Z───Z───Z───Z───Z───Z───·───·───Z───Z───Z───Z───Y───Z───·───·───·───·─┐
  │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░│
┌─·───·───·───·───Y───Z───Z───Z───Z───Z───Z───Z───Z───Z───Y───Y───Z───Z───Z───Z───·─┘
│░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│  
└─·───·───·───X───X───Z───Z───Z───Z───Z───Z───Z───Z───Z───Y───X───·───·───·───·───·─┐
  │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░░░│ 